In [ ]:
!pip install imblearn==0.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.6/235.6 kB 41.6 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from imblearn.over_sampling import RandomOverSampler
from sklearn.ensemble import StackingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, confusion_matrix, f1_score, recall_score

In [ ]:
# Load a dataset into a Pandas Dataframe.
df = pd.read_csv('/work/Dataset2.csv')

# Display the first 3 examples.
df.head(3)

,"He leído el consentimiento y las explicaciones del investigador, por lo que reconozco que al completar y entregar esta escala estoy dando mi consentimiento para participar en esta investigación .",Edad,Lugar de Procedencia,Estado civil,Nivel de estudios,Número de hijos menores,"1.- En general, ¿como describiría usted su relación de pareja?",2.- ¿Con cuanta dificultad usted y su pareja resuelven sus diferencias?,3.- ¿Al terminar las discusiones usted se siente decaída o mal con usted misma?,"4.- ¿Las discusiones terminan en golpes, patadas o empujones?",...,No toleré nada que no me permitiera continuar con lo que estaba haciendo,Sentí que estaba muy irritable,Depresion - Porcentaje,Ansiedad - Porcentaje,Estres - Porcentaje,Depresión,Ansiedad,Estres,Porcentaje Violencia,Violencia
0,Sí,23,Selva,Conviviente,Pregrado completo (Estudios universitarios),1,1,3,3,1,...,0,0,8,6,6,Normal,Normal,Normal,13,NO
1,Sí,22,Costa,Divorciada,Técnico completo,0,3,3,2,1,...,1,2,30,24,26,Extremadamente Grave,Extremadamente Grave,Grave,18,SI
2,Sí,29,Selva,Divorciada,Pregrado completo (Estudios universitarios),1,1,2,2,1,...,2,3,28,28,28,Extremadamente Grave,Extremadamente Grave,Grave,16,SI


In [ ]:
# Variables categoricas
# Selecciono las columnas que son categoricas
# Aplic dummies para convertir variables categoricas
categorical_columns = ['Lugar de Procedencia', 
                        'Estado civil', 
                        'Nivel de estudios',
                        'Actualmente, ¿recibe atención psicológica y/o psiquiátrica?',
                        'Actualmente, ¿cuenta con apoyo de familiares y/o amigos?',
                        'Departamento',
                        'Violencia'
                        ]
label_encoder = LabelEncoder()
 #Iterar a través de las columnas categóricas y aplicar label encoding a cada una
for column in categorical_columns:
    df[column] = label_encoder.fit_transform(df[column])

     # Obtener la asignación de números a valores únicos
    mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
    print(f'Mapping para la columna {column}: {mapping}')
df.head(20)

#df_encode = pd.get_dummies(data = df, columns = categorical_columns,
             # drop_first =True)
#df_encode.head(100)

Mapping para la columna Lugar de Procedencia: {'Costa': 0, 'Extranjero': 1, 'Selva': 2, 'Sierra': 3}
Mapping para la columna Estado civil: {'Casada': 0, 'Conviviente': 1, 'Divorciada': 2, 'Soltera': 3}
Mapping para la columna Nivel de estudios: {'Pregrado completo (Estudios universitarios)': 0, 'Primaria completa': 1, 'Secundaria completa': 2, 'Sin estudios (sabe leer y escribir)': 3, 'Técnico completo': 4}
Mapping para la columna Actualmente, ¿recibe atención psicológica y/o psiquiátrica?: {'No': 0, 'Si': 1}
Mapping para la columna Actualmente, ¿cuenta con apoyo de familiares y/o amigos?: {'No': 0, 'Si': 1, 'Sí': 2}
Mapping para la columna Departamento: {'Amazonas': 0, 'Apurímac': 1, 'Arequipa': 2, 'Ayacucho': 3, 'Cajamarca': 4, 'Callao': 5, 'Cusco': 6, 'Huancavelica': 7, 'Huánuco': 8, 'Ica': 9, 'Junín': 10, 'La Libertad': 11, 'Lambayeque': 12, 'Lima': 13, 'Loreto': 14, 'Madre de Dios': 15, 'Moquegua': 16, 'Pasco': 17, 'Piura': 18, 'Puno': 19, 'San Martín': 20, 'Tacna': 21, 'Tumbes': 

,"He leído el consentimiento y las explicaciones del investigador, por lo que reconozco que al completar y entregar esta escala estoy dando mi consentimiento para participar en esta investigación .",Edad,Lugar de Procedencia,Estado civil,Nivel de estudios,Número de hijos menores,"1.- En general, ¿como describiría usted su relación de pareja?",2.- ¿Con cuanta dificultad usted y su pareja resuelven sus diferencias?,3.- ¿Al terminar las discusiones usted se siente decaída o mal con usted misma?,"4.- ¿Las discusiones terminan en golpes, patadas o empujones?",...,No toleré nada que no me permitiera continuar con lo que estaba haciendo,Sentí que estaba muy irritable,Depresion - Porcentaje,Ansiedad - Porcentaje,Estres - Porcentaje,Depresión,Ansiedad,Estres,Porcentaje Violencia,Violencia
0,Sí,23,2,1,0,1,1,3,3,1,...,0,0,8,6,6,Normal,Normal,Normal,13,0
1,Sí,22,0,2,4,0,3,3,2,1,...,1,2,30,24,26,Extremadamente Grave,Extremadamente Grave,Grave,18,1
2,Sí,29,2,2,0,1,1,2,2,1,...,2,3,28,28,28,Extremadamente Grave,Extremadamente Grave,Grave,16,1
3,Sí,49,2,0,0,1,1,2,3,3,...,3,2,22,22,34,Grave,Extremadamente Grave,Extremadamente Grave,19,1
4,Sí,20,2,0,2,0,3,2,2,2,...,1,3,30,28,28,Extremadamente Grave,Extremadamente Grave,Grave,18,1
5,Sí,36,0,3,0,5,2,3,1,1,...,3,1,30,30,30,Extremadamente Grave,Extremadamente Grave,Grave,15,1
6,Sí,26,2,3,2,1,3,2,2,3,...,3,3,24,32,30,Grave,Extremadamente Grave,Grave,18,1
7,Sí,29,0,2,3,4,3,1,1,1,...,1,1,28,26,22,Extremadamente Grave,Extremadamente Grave,Moderado,18,1
8,Sí,20,3,1,2,3,3,2,3,2,...,3,3,22,32,32,Grave,Extremadamente Grave,Grave,18,1
9,Sí,31,0,1,1,6,1,2,2,3,...,1,1,30,34,24,Extremadamente Grave,Extremadamente Grave,Moderado,16,1


In [ ]:
columns_to_drop = ['He leído el consentimiento y las explicaciones del investigador, por lo que reconozco que al completar y entregar esta escala estoy dando mi consentimiento para participar en esta investigación .',
                    '1.- En general, ¿como describiría usted su relación de pareja?', 
                    '2.- ¿Con cuanta dificultad usted y su pareja resuelven sus diferencias?', 
                    '3.- ¿Al terminar las discusiones usted se siente decaída o mal con usted misma?', 
                    '4.- ¿Las discusiones terminan en golpes, patadas o empujones?', 
                    '5.- ¿Usted se siente aterrada por lo que su pareja pueda decir o hacer?', 
                    '6.- ¿Alguna vez su pareja ha abusado fisicamente de usted?', 
                    '7.- ¿Alguna vez su pareja ha abusado emocionalmente de usted?',
                    '8.- ¿Alguna vez su pareja ha abusado sexualmente de usted?',
                    'Porcentaje Violencia',
                    'Depresion - Porcentaje',
                    'Ansiedad - Porcentaje',
                    'Estres - Porcentaje',
                    'Depresión',
                    'Ansiedad',
                    'Estres',
                    ]
df = df.drop(columns_to_drop, axis=1)
df

,Edad,Lugar de Procedencia,Estado civil,Nivel de estudios,Número de hijos menores,"Actualmente, ¿recibe atención psicológica y/o psiquiátrica?","Actualmente, ¿cuenta con apoyo de familiares y/o amigos?",Cuantas parejas ha tenido,Departamento,No podía sentir ningún sentimiento positivo,...,Sentí que estaba al punto de pánico,Sentí los latidos de mi corazón a pesar de no haber hecho ningún esfuerzo físico,Tuve miedo sin razón,Reaccioné exageradamente en ciertas situaciones,Sentí que tenia muchos nervios,Noté que me agitaba,Se me hizo difícil relajarme,No toleré nada que no me permitiera continuar con lo que estaba haciendo,Sentí que estaba muy irritable,Violencia
0,23,2,1,0,1,0,2,1,5,0,...,0,0,1,1,0,0,0,0,0,0
1,22,0,2,4,0,0,2,3,23,1,...,3,2,1,2,2,2,2,1,2,1
2,29,2,2,0,1,0,0,8,6,2,...,3,2,2,3,1,1,2,2,3,1
3,49,2,0,0,1,0,0,1,24,1,...,1,1,1,3,2,3,3,3,2,1
4,20,2,0,2,0,0,2,6,13,3,...,3,2,3,1,1,3,2,1,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,43,0,2,2,4,0,0,6,10,1,...,3,1,1,3,1,1,1,3,1,1
995,48,3,1,2,6,1,0,7,2,2,...,2,1,2,1,2,2,3,3,3,1
996,30,2,3,3,0,0,0,7,19,0,...,0,0,1,2,0,0,2,1,2,0
997,49,2,2,4,2,0,0,1,9,3,...,1,3,1,1,1,1,2,3,1,1


In [ ]:
# Load a dataset into a Pandas Dataframe.
dfTest = pd.read_csv('/work/DataTestOriginal.csv')

# Display the first 3 examples.
dfTest.head(3)

,"He leído el consentimiento y las explicaciones del investigador, por lo que reconozco que al completar y entregar esta escala estoy dando mi consentimiento para participar en esta investigación .",Edad,Lugar de Procedencia,Zona,Estado civil,Nivel de estudios,Número de hijos menores,"1.- En general, ¿como describiría usted su relación de pareja?",2.- ¿Con cuanta dificultad usted y su pareja resuelven sus diferencias?,3.- ¿Al terminar las discusiones usted se siente decaída o mal con usted misma?,...,Sentí los latidos de mi corazón a pesar de no haber hecho ningún esfuerzo físico,Tuve miedo sin razón,Reaccioné exageradamente en ciertas situaciones,Sentí que tenia muchos nervios,Noté que me agitaba,Se me hizo difícil relajarme,No toleré nada que no me permitiera continuar con lo que estaba haciendo,Sentí que estaba muy irritable,Violencia Porcentaje,Violencia
0,Sí,23.0,Cierra,Rural,Casada,Pregrado completo (Estudios universitarios),1.0,1.0,2.0,3.0,...,1.0,1.0,0.0,2.0,1.0,1.0,2.0,0.0,13,NO
1,Sí,22.0,Costa,Urbano,Soltera,Técnico completo,0.0,2.0,2.0,2.0,...,1.0,1.0,0.0,1.0,0.0,1.0,1.0,2.0,12,NO
2,Sí,29.0,Sierra,Urbano,Casada,Pregrado completo (Estudios universitarios),1.0,1.0,2.0,2.0,...,2.0,1.0,1.0,0.0,1.0,2.0,0.0,2.0,12,NO


In [ ]:
# Variables categoricas
# Selecciono las columnas que son categoricas
# Aplic dummies para convertir variables categoricas
categorical_columns_test = ['Lugar de Procedencia', 
                        'Estado civil', 
                        'Nivel de estudios',
                        'Actualmente, ¿recibe atención psicológica y/o psiquiátrica?',
                        'Actualmente, ¿cuenta con apoyo de familiares y/o amigos?',
                        'Departamento',
                        'Violencia'
                        ]
label_encoder = LabelEncoder()
 #Iterar a través de las columnas categóricas y aplicar label encoding a cada una
for column in categorical_columns_test:
    dfTest[column] = label_encoder.fit_transform(dfTest[column])

     # Obtener la asignación de números a valores únicos
    mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
    print(f'Mapping para la columna {column}: {mapping}')
dfTest.head(20)

#df_encode = pd.get_dummies(data = df, columns = categorical_columns,
             # drop_first =True)
#df_encode.head(100)

Mapping para la columna Lugar de Procedencia: {'Cierra': 0, 'Costa': 1, 'Extranjero': 2, 'Selva': 3, 'Sierra': 4, nan: 5}
Mapping para la columna Estado civil: {'Casada': 0, 'Conviviente': 1, 'Divorciada': 2, 'Soltera': 3, nan: 4}
Mapping para la columna Nivel de estudios: {'Pregrado completo (Estudios universitarios)': 0, 'Primaria completa': 1, 'Secundaria completa': 2, 'Técnico completo': 3, nan: 4}
Mapping para la columna Actualmente, ¿recibe atención psicológica y/o psiquiátrica?: {'No': 0, 'Si': 1, nan: 2}
Mapping para la columna Actualmente, ¿cuenta con apoyo de familiares y/o amigos?: {'No': 0, 'Sí': 1, nan: 2}
Mapping para la columna Departamento: {'Amazonas': 0, 'Arequipa': 1, 'Ayacucho': 2, 'Cajamarca': 3, 'Callao': 4, 'Cusco': 5, 'Huánuco': 6, 'Ica': 7, 'Junín': 8, 'Lambayeque': 9, 'Lima': 10, 'Piura': 11, 'Puno': 12, 'San Martín': 13, 'Tacna': 14, 'Tumbes': 15, nan: 16}
Mapping para la columna Violencia: {'NO': 0, 'SI': 1}


,"He leído el consentimiento y las explicaciones del investigador, por lo que reconozco que al completar y entregar esta escala estoy dando mi consentimiento para participar en esta investigación .",Edad,Lugar de Procedencia,Zona,Estado civil,Nivel de estudios,Número de hijos menores,"1.- En general, ¿como describiría usted su relación de pareja?",2.- ¿Con cuanta dificultad usted y su pareja resuelven sus diferencias?,3.- ¿Al terminar las discusiones usted se siente decaída o mal con usted misma?,...,Sentí los latidos de mi corazón a pesar de no haber hecho ningún esfuerzo físico,Tuve miedo sin razón,Reaccioné exageradamente en ciertas situaciones,Sentí que tenia muchos nervios,Noté que me agitaba,Se me hizo difícil relajarme,No toleré nada que no me permitiera continuar con lo que estaba haciendo,Sentí que estaba muy irritable,Violencia Porcentaje,Violencia
0,Sí,23.0,0,Rural,0,0,1.0,1.0,2.0,3.0,...,1.0,1.0,0.0,2.0,1.0,1.0,2.0,0.0,13,0
1,Sí,22.0,1,Urbano,3,3,0.0,2.0,2.0,2.0,...,1.0,1.0,0.0,1.0,0.0,1.0,1.0,2.0,12,0
2,Sí,29.0,4,Urbano,0,0,1.0,1.0,2.0,2.0,...,2.0,1.0,1.0,0.0,1.0,2.0,0.0,2.0,12,0
3,Sí,49.0,4,Urbano,3,0,1.0,2.0,2.0,2.0,...,1.0,1.0,0.0,1.0,2.0,1.0,0.0,2.0,12,0
4,Sí,20.0,1,Urbano,3,2,0.0,1.0,2.0,2.0,...,1.0,1.0,2.0,2.0,1.0,1.0,2.0,2.0,10,0
5,Sí,22.0,1,NaN,3,3,0.0,1.0,1.0,1.0,...,0.0,0.0,2.0,1.0,0.0,1.0,1.0,1.0,8,0
6,Sí,27.0,1,NaN,1,2,2.0,2.0,2.0,3.0,...,2.0,0.0,3.0,3.0,0.0,2.0,2.0,2.0,17,1
7,Sí,35.0,4,NaN,3,1,2.0,1.0,2.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12,0
8,Sí,33.0,1,NaN,1,1,2.0,3.0,3.0,3.0,...,2.0,2.0,2.0,2.0,1.0,1.0,2.0,3.0,18,1
9,Sí,22.0,4,NaN,1,0,1.0,1.0,1.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,10,0


In [ ]:
columns_to_drop_test = ['He leído el consentimiento y las explicaciones del investigador, por lo que reconozco que al completar y entregar esta escala estoy dando mi consentimiento para participar en esta investigación .',
                    '1.- En general, ¿como describiría usted su relación de pareja?', 
                    '2.- ¿Con cuanta dificultad usted y su pareja resuelven sus diferencias?', 
                    '3.- ¿Al terminar las discusiones usted se siente decaída o mal con usted misma?', 
                    '4.- ¿Las discusiones terminan en golpes, patadas o empujones?', 
                    '5.- ¿Usted se siente aterrada por lo que su pareja pueda decir o hacer?', 
                    '6.- ¿Alguna vez su pareja ha abusado fisicamente de usted?', 
                    '7.- ¿Alguna vez su pareja ha abusado emocionalmente de usted?',
                    '8.- ¿Alguna vez su pareja ha abusado sexualmente de usted?',
                    'Violencia Porcentaje',
                    'Zona'
                    ]
dfTest = dfTest.drop(columns_to_drop_test, axis=1)
dfTest

,Edad,Lugar de Procedencia,Estado civil,Nivel de estudios,Número de hijos menores,"Actualmente, ¿recibe atención psicológica y/o psiquiátrica?","Actualmente, ¿cuenta con apoyo de familiares y/o amigos?",Cuantas parejas ha tenido,Departamento,No podía sentir ningún sentimiento positivo,...,Sentí que estaba al punto de pánico,Sentí los latidos de mi corazón a pesar de no haber hecho ningún esfuerzo físico,Tuve miedo sin razón,Reaccioné exageradamente en ciertas situaciones,Sentí que tenia muchos nervios,Noté que me agitaba,Se me hizo difícil relajarme,No toleré nada que no me permitiera continuar con lo que estaba haciendo,Sentí que estaba muy irritable,Violencia
0,23.0,0,0,0,1.0,0,1,1.0,10,1.0,...,0.0,1.0,1.0,0.0,2.0,1.0,1.0,2.0,0.0,0
1,22.0,1,3,3,0.0,0,1,1.0,10,0.0,...,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,2.0,0
2,29.0,4,0,0,1.0,0,0,2.0,10,1.0,...,2.0,2.0,1.0,1.0,0.0,1.0,2.0,0.0,2.0,0
3,49.0,4,3,0,1.0,0,0,4.0,10,2.0,...,2.0,1.0,1.0,0.0,1.0,2.0,1.0,0.0,2.0,0
4,20.0,1,3,2,0.0,0,1,3.0,10,1.0,...,1.0,1.0,1.0,2.0,2.0,1.0,1.0,2.0,2.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,35.0,4,0,0,0.0,0,1,1.0,10,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
81,59.0,4,0,1,4.0,0,1,1.0,10,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1
82,20.0,1,3,2,0.0,0,1,1.0,10,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
83,20.0,1,3,2,0.0,0,1,0.0,10,2.0,...,0.0,2.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0,0


In [ ]:
valores_nulos = dfTest.isnull()

# Visualizar qué filas tienen valores nulos
filas_con_nulos = dfTest[dfTest.isnull().any(axis=1)]

# Mostrar DataFrame con valores nulos
print("DataFrame con Valores Nulos:")
print(df)
print("\nValores Nulos:")
print(valores_nulos)
print("\nFilas con Valores Nulos:")
print(filas_con_nulos)

DataFrame con Valores Nulos:
     Edad  Lugar de Procedencia  Estado civil  Nivel de estudios  \
0      23                     2             1                  0   
1      22                     0             2                  4   
2      29                     2             2                  0   
3      49                     2             0                  0   
4      20                     2             0                  2   
..    ...                   ...           ...                ...   
994    43                     0             2                  2   
995    48                     3             1                  2   
996    30                     2             3                  3   
997    49                     2             2                  4   
998    47                     2             0                  3   

     Número de hijos menores  \
0                          1   
1                          0   
2                          1   
3                         

In [ ]:
# Eliminar filas con valores nulos
dfTest = dfTest.dropna()

# Mostrar DataFrame sin valores nulos
#print("\nDataFrame sin Valores Nulos:")
#print(df_sin_nulos)
dfTest.isnull().sum()

Edad                                                                                                        0
Lugar de Procedencia                                                                                        0
Estado civil                                                                                                0
Nivel de estudios                                                                                           0
Número de hijos menores                                                                                     0
Actualmente, ¿recibe atención psicológica y/o psiquiátrica?                                                 0
Actualmente, ¿cuenta con apoyo de familiares y/o amigos?                                                    0
Cuantas parejas ha tenido                                                                                   0
Departamento                                                                                                0
No podía s

In [ ]:
XTest = dfTest.drop(['Edad',
			'Lugar de Procedencia',
			'Estado civil',
			'Nivel de estudios',
			'Número de hijos menores',
			'Actualmente, ¿recibe atención psicológica y/o psiquiátrica?',
			'Actualmente, ¿cuenta con apoyo de familiares y/o amigos?',
			'Cuantas parejas ha tenido',
			'Departamento',
			'No podía sentir ningún sentimiento positivo',
			'Me di cuenta que tenia la boca seca',
			'No me pude entusiasmar por nada',
			'Estaba preocupado por situaciones en las cuales podía tener pánico o en las que podría hacer el ridículo',
			'Sentí que estaba al punto de pánico',
			'Reaccioné exageradamente en ciertas situaciones',
			'Se me hizo difícil relajarme',
			'Sentí que estaba muy irritable',
			'Violencia'
			], axis = 1)
yTest = dfTest['Violencia']        

In [ ]:
print("Media: \n", np.mean(df))
print("Mediana: \n", np.median(df))
print("Desviación Estándar: \n", np.std(df))
print("Variancia: \n", np.var(df))

Media: 
 Edad                                                                                                        33.898899
Lugar de Procedencia                                                                                         1.538539
Estado civil                                                                                                 1.483483
Nivel de estudios                                                                                            1.943944
Número de hijos menores                                                                                      2.958959
Actualmente, ¿recibe atención psicológica y/o psiquiátrica?                                                  0.479479
Actualmente, ¿cuenta con apoyo de familiares y/o amigos?                                                     0.505506
Cuantas parejas ha tenido                                                                                    4.001001
Departamento                                   

In [ ]:
#Seteamos nuestas variables independientes
X = df.drop(['Violencia'], axis = 1)
#Seteamos nuestra variable target dependiente
y = df['Violencia']

In [ ]:
#X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

In [ ]:
clf_feature_selection = RandomForestClassifier(n_estimators=100, random_state=42)
sfm = SelectFromModel(clf_feature_selection)
X_selected = sfm.fit_transform(X, y)
selected_features = X.columns[sfm.get_support()]
selected_features

Index(['Me costó mucho relajarme',
       'Se me hizo difícil tomar la iniciativa para hacer cosas',
       'Sentí que no tenia nada por que vivir', 'Me sentí triste y deprimido',
       'Sentí que valía muy poco como persona',
       'Sentí que la vida no tenia ningún sentido',
       'Se me hizo difícil respirar', 'Sentí que mis manos temblaban',
       'Sentí los latidos de mi corazón a pesar de no haber hecho ningún esfuerzo físico',
       'Tuve miedo sin razón', 'Sentí que tenia muchos nervios',
       'Noté que me agitaba',
       'No toleré nada que no me permitiera continuar con lo que estaba haciendo'],
      dtype='object')

In [ ]:
# Numero de arboles del Random Forest
n_estimators = [int (x) for x in np.linspace(start = 10, stop = 150, num = 10)]
# Numero de caracteristicas a considerar
max_features = ['auto', 'sqrt']
# Numero maximo de niveles en arbol
max_depth = [2, 4, 6, 8]
#Muestra minima para dividir un nodo
min_samples_split = [2, 5]
#Muestra minima que requiere una hoja de nodo
min_samples_leaf = [1, 2, 3, 4]
#Metodo para seleccionar las muestras
bootstrap = [True, False]

In [ ]:
# Creación del grid de parametros para random forest
rf_param_grid = {'n_estimators':n_estimators,
                'max_features': max_features,
                'max_depth': max_depth,
                'min_samples_split': min_samples_split,
                'min_samples_leaf': min_samples_leaf,
                'bootstrap': bootstrap
                }
print(rf_param_grid)

{'n_estimators': [10, 25, 41, 56, 72, 87, 103, 118, 134, 150], 'max_features': ['auto', 'sqrt'], 'max_depth': [2, 4, 6, 8], 'min_samples_split': [2, 5], 'min_samples_leaf': [1, 2, 3, 4], 'bootstrap': [True, False]}


In [ ]:
rf_Model = RandomForestClassifier()


In [ ]:
rf_Grid = GridSearchCV(estimator=rf_Model, param_grid= rf_param_grid, cv= 3, verbose= 2, n_jobs= 4)

In [ ]:
rf_Grid.fit(X_selected, y)

[CV] END bootstrap=False, max_depth=6, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=118; total time=   0.6s
[CV] END bootstrap=False, max_depth=6, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=118; total time=   0.5s
[CV] END bootstrap=False, max_depth=6, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=118; total time=   0.6s
[CV] END bootstrap=False, max_depth=6, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=134; total time=   0.5s
[CV] END bootstrap=False, max_depth=6, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=134; total time=   0.5s
[CV] END bootstrap=False, max_depth=6, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=150; total time=   0.5s
[CV] END bootstrap=False, max_depth=6, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=10; total time=   0.0s
[CV] END bootstrap=False, max_depth=6, max_featur

GridSearchCV(cv=3, estimator=RandomForestClassifier(), n_jobs=4,
             param_grid={'bootstrap': [True, False], 'max_depth': [2, 4, 6, 8],
                         'max_features': ['auto', 'sqrt'],
                         'min_samples_leaf': [1, 2, 3, 4],
                         'min_samples_split': [2, 5],
                         'n_estimators': [10, 25, 41, 56, 72, 87, 103, 118, 134,
                                          150]},
             verbose=2)

In [ ]:
dfCV = pd.DataFrame(rf_Grid.cv_results_)
dfCV[[
'param_n_estimators',
'param_max_features',
'param_max_depth',
'param_min_samples_split',
'param_min_samples_leaf',
'param_bootstrap',
'mean_test_score'
]]

,param_n_estimators,param_max_features,param_max_depth,param_min_samples_split,param_min_samples_leaf,param_bootstrap,mean_test_score
0,10,auto,2,2,1,True,0.923924
1,25,auto,2,2,1,True,0.940941
2,41,auto,2,2,1,True,0.941942
3,56,auto,2,2,1,True,0.931932
4,72,auto,2,2,1,True,0.927928
...,...,...,...,...,...,...,...
1275,87,sqrt,8,5,4,False,0.993994
1276,103,sqrt,8,5,4,False,0.993994
1277,118,sqrt,8,5,4,False,0.993994
1278,134,sqrt,8,5,4,False,0.994995


In [ ]:
rf_Grid.best_params_

{'bootstrap': True,
 'max_depth': 8,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 10}

In [ ]:
print(f'Best Score - : {rf_Grid.best_score_:.3f}')
print(f'Best Params - : {rf_Grid.best_params_}')

Best Score - : 1.000
Best Params - : {'bootstrap': True, 'max_depth': 8, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 10}


In [ ]:
print(f'Accuracy - : {rf_Grid.score(XTest, yTest):.3f}')

Accuracy - : 0.821
/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


In [ ]:
base_classifiers = []

In [ ]:
# Crear el mejor modelo Random Forest
best_rf_model = RandomForestClassifier(**rf_Grid.best_params_)

# Agregar el mejor modelo Random Forest a la lista de clasificadores base
base_classifiers.append(('rf', best_rf_model))

## Support Vector Machine (SVM)

In [ ]:
# Control de penalización por error de clasificación
C = [1, 5, 10, 50, 100]
# Especifica el tipo de núcleo a utilizar en la función de decisión
kernel = ['linear', 'poly', 'rbf', 'sigmoid', 'precompute']
# Este parámetro especifica el grado del polinomio en el kernel polinómico.
degree = [1, 2, 3, 4, 5, 6]
#define cuánta influencia tiene un solo ejemplo de entrenamiento.
gamma = ['scale', 'auto']
# Asignar diferentes pesos a las clases
class_weight = ['dict', 'balanced']

In [ ]:
# Creación del grid de parametros
svm_param_grid = {'C': C,
                'kernel': kernel,
                'degree': degree,
                'gamma': gamma,
                'class_weight': class_weight
                }
print(svm_param_grid)

{'C': [1, 5, 10, 50, 100], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid', 'precompute'], 'degree': [1, 2, 3, 4, 5, 6], 'gamma': ['scale', 'auto'], 'class_weight': ['dict', 'balanced']}


In [ ]:
svm_Model = SVC()

In [ ]:
svm_Grid = GridSearchCV(estimator=svm_Model, param_grid= svm_param_grid, cv= 3, verbose= 2, n_jobs= 4)

In [ ]:
svm_Grid.fit(X_selected, y)

[CV] END C=10, class_weight=balanced, degree=3, gamma=scale, kernel=rbf; total time=   0.0s
[CV] END C=10, class_weight=balanced, degree=3, gamma=scale, kernel=rbf; total time=   0.1s
[CV] END C=10, class_weight=balanced, degree=1, gamma=scale, kernel=sigmoid; total time=   0.1s
[CV] END C=10, class_weight=balanced, degree=1, gamma=scale, kernel=precompute; total time=   0.0s
[CV] END C=10, class_weight=balanced, degree=1, gamma=scale, kernel=precompute; total time=   0.0s
[CV] END C=10, class_weight=balanced, degree=1, gamma=scale, kernel=precompute; total time=   0.0s
[CV] END C=10, class_weight=balanced, degree=3, gamma=auto, kernel=sigmoid; total time=   0.2s
[CV] END C=10, class_weight=balanced, degree=1, gamma=auto, kernel=linear; total time=   0.0s
[CV] END C=10, class_weight=balanced, degree=1, gamma=auto, kernel=linear; total time=   0.0s
[CV] END C=10, class_weight=balanced, degree=1, gamma=auto, kernel=linear; total time=   0.1s
[CV] END C=10, class_weight=balanced, degree=2

GridSearchCV(cv=3, estimator=SVC(), n_jobs=4,
             param_grid={'C': [1, 5, 10, 50, 100],
                         'class_weight': ['dict', 'balanced'],
                         'degree': [1, 2, 3, 4, 5, 6],
                         'gamma': ['scale', 'auto'],
                         'kernel': ['linear', 'poly', 'rbf', 'sigmoid',
                                    'precompute']},
             verbose=2)

In [ ]:
dfCVSVM = pd.DataFrame(svm_Grid.cv_results_)
dfCVSVM[[
'param_C',
'param_kernel',
'param_degree',
'param_gamma',
'param_class_weight',
'mean_test_score'
]]

,param_C,param_kernel,param_degree,param_gamma,param_class_weight,mean_test_score
0,1,linear,1,scale,dict,NaN
1,1,poly,1,scale,dict,NaN
2,1,rbf,1,scale,dict,NaN
3,1,sigmoid,1,scale,dict,NaN
4,1,precompute,1,scale,dict,NaN
...,...,...,...,...,...,...
595,100,linear,6,auto,balanced,0.981982
596,100,poly,6,auto,balanced,0.987988
597,100,rbf,6,auto,balanced,0.985986
598,100,sigmoid,6,auto,balanced,0.010010


In [ ]:
svm_Grid.best_params_

{'C': 5,
 'class_weight': 'balanced',
 'degree': 1,
 'gamma': 'scale',
 'kernel': 'rbf'}

In [ ]:
print(f'Best Score - : {svm_Grid.best_score_:.3f}')
print(f'Best Params - : {svm_Grid.best_params_}')

Best Score - : 0.990
Best Params - : {'C': 5, 'class_weight': 'balanced', 'degree': 1, 'gamma': 'scale', 'kernel': 'rbf'}


In [ ]:
print(f'Accuracy - : {svm_Grid.score(XTest, yTest):.3f}')

Accuracy - : 0.821
/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but SVC was fitted without feature names
  warnings.warn(


In [ ]:
# Crear el mejor modelo SVM después de la búsqueda de hiperparámetros
svm_Model = SVC(**svm_Grid.best_params_)
best_svm_model = svm_Model

# Agregar el mejor modelo SVM a la lista de clasificadores base
base_classifiers.append(('svm', best_svm_model))

## Neural Network (NN)

In [ ]:
def create_model_nn(neurons, optimizer):
    nn_model = Sequential()
    nn_model.add(Dense(neurons, input_dim=13, activation='relu'))
    nn_model.add(Dropout(0.2))
    nn_model.add(Dense(1, activation='sigmoid'))  # Una neurona con activación sigmoide para clasificación binaria
    nn_model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return nn_model

In [ ]:
nn_param_grid = {'neurons': [2, 8, 16],
                    'batch_size': [4, 16],
                    'optimizer': ['SGD', 'RMSprop', 'Adam']}

In [ ]:
nn_model = KerasClassifier(build_fn=create_model_nn, epochs=10, verbose=1) 

/tmp/ipykernel_40/1422491447.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  nn_model = KerasClassifier(build_fn=create_model_nn, epochs=10, verbose=1)


In [ ]:
nn_grid = GridSearchCV(estimator=nn_model, param_grid=nn_param_grid, n_jobs=1, cv=3)

In [ ]:
nn_grid_results = nn_grid.fit(X_selected, y)

Epoch 5/10
167/167 [==============================] - 0s 1ms/step - loss: 0.5013 - accuracy: 0.6862
Epoch 6/10
167/167 [==============================] - 0s 1ms/step - loss: 0.4952 - accuracy: 0.7027
Epoch 7/10
167/167 [==============================] - 0s 1ms/step - loss: 0.4953 - accuracy: 0.6727
Epoch 8/10
167/167 [==============================] - 0s 1ms/step - loss: 0.4882 - accuracy: 0.6757
Epoch 9/10
167/167 [==============================] - 0s 1ms/step - loss: 0.4778 - accuracy: 0.6772
Epoch 10/10
84/84 [==============================] - 0s 856us/step - loss: 0.3919 - accuracy: 0.8318
Epoch 1/10
167/167 [==============================] - 1s 1ms/step - loss: 1.6349 - accuracy: 0.4114
Epoch 2/10
167/167 [==============================] - 0s 1ms/step - loss: 0.6847 - accuracy: 0.6967
Epoch 3/10
167/167 [==============================] - 0s 1ms/step - loss: 0.6418 - accuracy: 0.7387
Epoch 4/10
167/167 [==============================] - 0s 1ms/step - loss: 0.6213 - accuracy: 0.7553

In [ ]:
print(f'Best Score - : {nn_grid_results.best_score_:.3f}')
print(f'Best Params - : {nn_grid_results.best_params_}')

Best Score - : 0.913
Best Params - : {'batch_size': 4, 'neurons': 16, 'optimizer': 'RMSprop'}


In [ ]:
print(f'Accuracy - : {nn_grid_results.score(XTest, yTest):.3f}')

21/21 [==============================] - 0s 931us/step - loss: 0.5692 - accuracy: 0.7619
Accuracy - : 0.762


In [ ]:
# Crear el mejor modelo de la red neuronal
best_nn_model = KerasClassifier(build_fn=create_model_nn, epochs=10, verbose=1, **nn_grid_results.best_params_)

# Agregar el mejor modelo de la red neuronal a la lista de clasificadores base
base_classifiers.append(('nn', best_nn_model))

/tmp/ipykernel_40/127405060.py:2: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  best_nn_model = KerasClassifier(build_fn=create_model_nn, epochs=10, verbose=1, **nn_grid_results.best_params_)


## Stacked Ensemble

In [ ]:
#Utilizamos RandomOverSampler de la bibiloteca imblearn
oversample = RandomOverSampler(sampling_strategy='minority')
X_oversampled, y_oversampled = oversample.fit_resample(X_selected, y)

In [ ]:
clf_stacking = StackingClassifier(estimators=base_classifiers, final_estimator= LogisticRegression())

#skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# Evaluar el modelo utilizando validación cruzada de 3 pliegues:
#cv_results = cross_val_score(clf_stacking, X_oversampled, y_oversampled, cv=skf, scoring='roc_auc')

# Medir el desempeño del modelo
#print("AUC: %.3f (%.3f)" % (np.mean(cv_results), np.std(cv_results)))

In [ ]:
clf_stacking.fit(X_selected, y)

StackingClassifier(estimators=[('rf',
                                RandomForestClassifier(max_depth=8,
                                                       n_estimators=10)),
                               ('svm',
                                SVC(C=5, class_weight='balanced', degree=1))],
                   final_estimator=LogisticRegression())

In [ ]:
clf_stacking.score(XTest, yTest)

/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but SVC was fitted without feature names
  warnings.warn(


0.8452380952380952

In [ ]:
predictions = clf_stacking.predict(X_selected)
predictions

array([0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0,
       0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
       1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1,
       1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0,
       0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1,
       0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1,

In [ ]:
print("Accuracy: ", accuracy_score(y, predictions))
print("Precision: ", precision_score(y, predictions))
print("Recall: ", recall_score(y, predictions))


#print("Confusion Matrix:\n", confusion_matrix(yTest, predictions, labels=[0,1]))

Accuracy:  0.997997997997998
Precision:  0.9986357435197817
Recall:  0.9986357435197817
F1 Score:  0.9986357435197817


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=d6fcd0a7-f1b9-4dea-9775-1ca2482fd900' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>